In [19]:
%matplotlib

Using matplotlib backend: MacOSX


In [1]:
from pathlib import Path
import re
from pprint import pprint

import numpy as np
from scipy import signal, ndimage
import matplotlib.pyplot as plt
from matplotlib import pyplot

import emd
import eelbrain
import mne
import trftools

## How to get the IF from the envelope, and turn it into the TRF?

1. Load in the envelope
2. Emd sift the envelope
3. Save the IF value
4. Pack the IF value with the sensor
5. Calculate the IF's TRF

In [3]:
## NATIVES ##

STIMULI = [str(i) for i in range(1, 13)]
DATA_ROOT = Path("/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results")  #Path("~").expanduser() / 'Data' / 'Alice'
PREDICTOR_audio_DIR = DATA_ROOT / 'TRFs_pridictors/audio_predictors'
PREDICTOR_word_DIR = DATA_ROOT / 'TRFs_pridictors/word_predictors'
EEG_DIR = DATA_ROOT / 'EEG_Natives' / 'Alice_natives_ICAed_fif'
Native_SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'S\d*', path.name)]
# Define a target directory for TRF estimates and make sure the directory is created
TRF_DIR = DATA_ROOT / 'TRFs_Natives'
TRF_DIR.mkdir(exist_ok=True)
print(Native_SUBJECTS)
print(len(Native_SUBJECTS))

['S13_Alice-natives_sfreq-100_raw.fif', 'S14_Alice-natives_sfreq-100_raw.fif', 'S12_Alice-natives_sfreq-100_raw.fif', 'S15_Alice-natives_sfreq-100_raw.fif', 'S11_Alice-natives_sfreq-100_raw.fif', 'S16_Alice-natives_sfreq-100_raw.fif', 'S17_Alice-natives_sfreq-100_raw.fif', 'S18_Alice-natives_sfreq-100_raw.fif', 'S19_Alice-natives_sfreq-100_raw.fif', 'S20_Alice-natives_sfreq-100_raw.fif', 'S21_Alice-natives_sfreq-100_raw.fif', 'S01_Alice-natives_sfreq-100_raw.fif', 'S03_Alice-natives_sfreq-100_raw.fif', 'S04_Alice-natives_sfreq-100_raw.fif', 'S05_Alice-natives_sfreq-100_raw.fif', 'S06_Alice-natives_sfreq-100_raw.fif', 'S08_Alice-natives_sfreq-100_raw.fif', 'S10_Alice-natives_sfreq-100_raw.fif', 'S22_Alice-natives_sfreq-100_raw.fif', 'S25_Alice-natives_sfreq-100_raw.fif', 'S26_Alice-natives_sfreq-100_raw.fif', 'S34_Alice-natives_sfreq-100_raw.fif', 'S35_Alice-natives_sfreq-100_raw.fif', 'S36_Alice-natives_sfreq-100_raw.fif', 'S37_Alice-natives_sfreq-100_raw.fif', 'S38_Alice-natives_sfreq

In [20]:
# Load the broad-band envelope and process it in the same way
envelope = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-1.pickle') for stimulus in STIMULI]  # Load in the data
"""# Code Explanation
envelopeL = []
for stimulus in STIMULI:
    envelope = eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-1.pickle')
    print(envelope)
    envelopeL.append(envelope)
"""

envelope = [x.bin(0.01, dim='time', label='start') for x in envelope]
#print(envelope)
""" # Code Explanation
envelopeL_2 = []
for x in envelope:
    envelope = x.bin(0.01, dim='time', label='start')
    envelopeL_2.append(envelope)
"""
envelope = [trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='envelope') for x in envelope]
#print(envelope)
#dir(envelope[1])

print(envelope[1].get_data())

plt.plot(envelope[1].get_data())

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
"""
# Try
env_x_AllLIST = []
for env_x in envelope:
    env_xLIST = []
    for i in range(len(env_x)):
        x = env_x[i]
        print(x)
    print()
    print()
        #env_xLIST.append(x)
    #env_x_AllLIST.append(env_xLIST)
#print(env_x_AllLIST)
"""

In [24]:
sample_rate = 100
imf, mask_freqs = emd.sift.mask_sift(envelope[1].get_data(), ret_mask_freq=True, max_imfs=6)  # delete >>, mask_freqs=30/sample_rate

# xy = envelope data 
print(mask_freqs * sample_rate)

emd.plotting.plot_imfs(imf)

# 每個ＩＭＦ的順時頻率都做一個TRF 去跑統計
"""
[18.86807686  9.43403843  4.71701922  2.35850961  1.1792548   0.5896274 ]>> = 每個IMF最多的頻率
4hz>> close to the syllables, 18 hz >> close to phoneme speed, but normally we consider syllables as speed
>> use the third IMF >> 
# IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'hilbert') get the IF
"""

[18.86807686  9.43403843  4.71701922  2.35850961  1.1792548   0.5896274 ]


''